# Ex 1

Pe baza definiţiei unui model Markov ascuns, demonstraţi formula recurenţei pentru variabila forward de la
pagina 23 a cursului 4. Încărcaţi argumentul fie în Markdown/Latex, fie ca foto la foia de lucru.

Avem de demonstrat formula $\alpha_t(i) = P(o_1, o_2, \dots, o_t, S_t = q_i \mid \lambda)$.

Observam ca $\alpha_t(i)$ este probabilitatea partiala (forward) a observarii secventei $(o_1, o_2, \dots, o_t)$ si a ajungerii in starea $q_i$ in timpul $t$, conditionata de modelul $\lambda$.

Vom demonstra formula prin recurenta:

1. Pentru $t = 1$, $\alpha_t(i)$ este probabilitatea stistemului de a incepe in starea $q_i$ si a genera observatia $o_1$:

$$\alpha_1(i) = P(o_1, S_1 = q_i \mid \lambda) = P(S_1 = q_i \mid \lambda) \cdot P(o_1 \mid S_1 = q_i, \lambda)$$

Cum $P(S_1 = q_i \mid \lambda) = \pi_i$ si $P(o_1 \mid S_1 = q_i, \lambda) = b_i(o_1)$, avem:

$$\alpha_1(i) = \pi_i \cdot b_i(o_1)$$

2. Pentrt $t > 1$, folosim independenta conditionata din modelul Markov ascuns pt a exprima $\alpha_t(i)$ in functie de valorile anterioare $\alpha_{t-1}(j)$:

$$\alpha_t(i) = P(o_1, o_2, \dots, o_t, S_t = q_i \mid \lambda)$$

Exprimam probabilitatea ca suma peste toate starile posibile ale timpului $t - 1$ (notate $q_j$), deoarece $S_t$ depinde doar de $S_{t-1}$:

$$\alpha_t(i) = \sum_{j=1}^{N} P(o_1, o_2, \dots, o_t, S_{t-1} = q_j, S_t = q_i \mid \lambda)$$

Cum $P(o_1, o_2, \dots, o_{t-1}, S_{t-1} = q_j \mid \lambda)$ este tocmai $\alpha_{t-1}(j)$, inlocuim si avem:

$$\alpha_{t}(i) = \sum_{j=1}^{N} \alpha_{t-1}(j) \cdot a_{ji} \cdot b_{i}(o_{t})$$

Aceasta e formula recursiva pt $\alpha_{t}(i)$.

# Ex 2

Reluaţi exerciţiul de la laborator, de data aceasta implementând direct alogritmul Viterbi descris în cursul 4. Mai
jos redăm enunţul acestuia:

Un profesor dă teste care pot fi dificile, medii sau uşoare. Probabilitatea dificultăţii primului test este aceeaşi. Dacă, la
un moment dat, dă un test dificil, următorul test poate fi doar mediu sau uşor, cu aceeaşi probabilitate. Însă dacă dă un
test mediu sau uşor, atunci următorul test va fi dificil cu probabilitate 0.5, sau mediu sau uşor cu aceeaşi probabilitate,
0.25.

Nota unui student la test, FB, B, S sau NS depinde de dificultatea testului. Astfel, probabilităţile condiţionate ale notei
obţinute, dată fiind dificultatea testului, sunt:

- test dificil: [0.1, 0.2, 0.4, 0.3];

- test mediu: [0.15, 0.25, 0.5, 0.1];

- test uşor: [0.2, 0.3, 0.4, 0.1].

Să presupunem că aţi observat următoarea secvenţă de note: FB, FB, S, B, B, S, B, B, NS, B, B, S. Determinaţi cea mai
probabilă secvenţă de dificultăţi pentru testele corespunzaătoare şi probabilitatea acesteia.

In [7]:
import numpy as np

states = ['Dificil', 'Mediu', 'Usor']
observations = ['FB', 'B', 'S', 'NS']
obs_sequence = ['FB', 'FB', 'S', 'B', 'B', 'S', 'B', 'B', 'NS', 'B', 'B', 'S']

start_prob = np.array([1/3, 1/3, 1/3])

trans_prob = {
    'Dificil': np.array([0, 0.5, 0.5]),
    'Mediu': np.array([0.5, 0.25, 0.25]),
    'Usor': np.array([0.5, 0.25, 0.25])
}

emit_prob = {
    'Dificil': np.array([0.1, 0.2, 0.4, 0.3]),
    'Mediu': np.array([0.15, 0.25, 0.5, 0.1]),
    'Usor': np.array([0.2, 0.3, 0.4, 0.1])
}

T = len(obs_sequence)
N = len(states)
delta = np.zeros((T, N))
psi = np.zeros((T, N), dtype=int)

for i, state in enumerate(states):
    delta[0, i] = start_prob[i] * emit_prob[state][observations.index(obs_sequence[0])]

for t in range(1, T):
    for j, next_state in enumerate(states):
        max_prob, max_state = max(
            (delta[t-1, i] * trans_prob[states[i]][j], i) for i in range(N)
        )
        delta[t, j] = max_prob * emit_prob[next_state][observations.index(obs_sequence[t])]
        psi[t, j] = max_state

final_prob, final_state = max((delta[T-1, i], i) for i in range(N))

best_path = [final_state]
for t in range(T-1, 0, -1):
    best_path.append(psi[t, best_path[-1]])

best_path.reverse()
state_sequence = [states[i] for i in best_path]

print("Cea mai probabilă secvență de dificultăți:", state_sequence)
print("Probabilitatea acestei secvențe:", final_prob)


Cea mai probabilă secvență de dificultăți: ['Usor', 'Usor', 'Dificil', 'Usor', 'Dificil', 'Mediu', 'Dificil', 'Usor', 'Dificil', 'Usor', 'Dificil', 'Mediu']
Probabilitatea acestei secvențe: 2.1093750000000005e-11
